# TPS-Aug-2022

In [1]:
class Config:
    NB = '208'
    dataset_NB = '108'

    raw_data_dir = '../data/raw/'
    processed_data_dir = '../data/processed/'
    interim_dir = '../data/interim/'
    submission_dir = '../data/submission/'

    random_seed = 42
    n_folds = 5

    row_id = 'id'
    target = 'failure'

## Import libralies

In [2]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

plotly_template = dict(
    layout=go.Layout(
        template='plotly_dark',
        font=dict(
            family="Franklin Gothic",
            size=12
        ),
        height=500,
        width=1000,
    )
)


color_palette = {
    'Bin': ['#016CC9','#E876A3'],
    'Cat5': ['#E876A3', '#E0A224', '#63B70D', '#6BCFF6', '#13399E'],
}

In [4]:
import random
import joblib
import itertools
from itertools import combinations

from sklearn.model_selection import StratifiedKFold, GroupKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

from sklearn.metrics import roc_auc_score, roc_curve, auc
from lightgbm import LGBMClassifier, early_stopping

## Load and check data

In [5]:
df_train = pd.read_pickle(Config.processed_data_dir + f'nb{Config.dataset_NB}_train.pkl', compression='zip')
df_test = pd.read_pickle(Config.processed_data_dir + f'nb{Config.dataset_NB}_test.pkl', compression='zip')

submission = pd.read_csv(Config.raw_data_dir + 'sample_submission.csv', header=None)

df_train.shape

(26570, 35)

In [6]:
df_train.head()

,id,product_code,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,missing_loading,missing_measurement_3,missing_measurement_4,missing_measurement_5,missing_measurement_9,ohe0_5,ohe0_7,ohe1_5,ohe1_6,ohe1_7,ohe1_8
0,0,A,80.10,9,5,7.0,8.0,11.0,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,14.537333,13.034,14.684,764.100,0.0,False,False,False,False,False,0.0,1.0,0.0,0.0,0.0,1.0
1,1,A,84.89,9,5,14.0,3.0,11.0,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425000,14.395,15.631,682.057,0.0,False,False,False,False,False,0.0,1.0,0.0,0.0,0.0,1.0
2,2,A,82.43,9,5,12.0,1.0,11.0,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,19.391,13.798,16.711,18.631000,14.094,17.946,663.376,0.0,False,False,False,False,False,0.0,1.0,0.0,0.0,0.0,1.0
3,3,A,101.07,9,5,13.0,2.0,11.0,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562000,16.154,17.172,826.282,0.0,False,False,False,False,False,0.0,1.0,0.0,0.0,0.0,1.0
4,4,A,188.06,9,5,9.0,2.0,11.0,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760000,13.153,16.412,579.885,0.0,False,False,False,False,False,0.0,1.0,0.0,0.0,0.0,1.0


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26570 entries, 0 to 26569
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     26570 non-null  int64  
 1   product_code           26570 non-null  object 
 2   loading                26570 non-null  float64
 3   attribute_2            26570 non-null  int64  
 4   attribute_3            26570 non-null  int64  
 5   measurement_0          26570 non-null  float64
 6   measurement_1          26570 non-null  float64
 7   measurement_2          26570 non-null  float64
 8   measurement_3          26570 non-null  float64
 9   measurement_4          26570 non-null  float64
 10  measurement_5          26570 non-null  float64
 11  measurement_6          26570 non-null  float64
 12  measurement_7          26570 non-null  float64
 13  measurement_8          26570 non-null  float64
 14  measurement_9          26570 non-null  float64
 15  me

In [8]:
def seed_everything(seed):

    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [9]:
'''# Category変数のLabelEncoding
cat_features = [
    'product_code'
]
cat_features = [f"{cf}_last" for cf in cat_features]
for cat_col in cat_features:
    encoder = LabelEncoder()
    df_train[cat_col] = encoder.fit_transform(df_train[cat_col])
    df_test[cat_col] = encoder.transform(df_test[cat_col])
'''

'# Category変数のLabelEncoding\ncat_features = [\n    \'product_code\'\n]\ncat_features = [f"{cf}_last" for cf in cat_features]\nfor cat_col in cat_features:\n    encoder = LabelEncoder()\n    df_train[cat_col] = encoder.fit_transform(df_train[cat_col])\n    df_test[cat_col] = encoder.transform(df_test[cat_col])\n'

In [10]:
# Get feature list
features = [col for col in df_train.columns if col not in [Config.row_id, Config.target, 'product_code']]

# Get parameter list
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'seed': Config.random_seed,
    'n_estimators': 20000,
    #'max_depth': 5,
    #'num_leaves': 30,
    'learning_rate': 0.01,
    #'feature_fraction': 0.20,
    #'bagging_freq': 10,
    #'bagging_fraction': 0.50,
    'n_jobs': -1,
    #'lambda_l2': 2,
    #'min_data_in_leaf': 40,
}

callbacks = [lgb.early_stopping(50), lgb.log_evaluation(500)]

In [11]:
# 約10秒

# Create a numpy array to store test predictions
test_predictions = np.zeros(len(df_test))

# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(df_train))

feature_importance_df = pd.DataFrame(index=features)
y_valids, val_preds =[],[]
amex_scores = []

#kfold = StratifiedKFold(n_splits = Config.n_folds, shuffle = True, random_state = Config.random_seed)
#for fold, (train_idx, valid_idx) in enumerate(kfold.split(df_train, df_train[Config.target])):
kfold = GroupKFold(n_splits=Config.n_folds) # must be 5 because of the 5 product codes
for fold, (train_idx, valid_idx) in enumerate(kfold.split(df_train, df_train[Config.target], df_train['product_code'])):

    print(' ')
    print('-'*50)
    print(f'Training fold {fold+1} with {len(features)} features...')

    x_train, x_val = df_train[features].iloc[train_idx], df_train[features].iloc[valid_idx]
    y_train, y_val = df_train[Config.target].iloc[train_idx], df_train[Config.target].iloc[valid_idx]
    # lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=cat_features)
    # lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature=cat_features)
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_valid = lgb.Dataset(x_val, y_val)

    model = lgb.train(params=params, train_set=lgb_train, valid_sets=[lgb_train, lgb_valid], valid_names=['train', 'valid'], callbacks=callbacks)
    print(f'================================== training {fold+1} fin. ==================================')

    # Predict validation data
    print(f'================================== validation-data predicting ... ==================================')
    val_pred = model.predict(x_val)
    oof_predictions[valid_idx] = val_pred

    # Predict test data
    print(f'================================== test-data predicting ... ==================================')
    test_pred = model.predict(df_test[features])
    test_predictions += test_pred / Config.n_folds

    # save results
    y_valids.append(y_val)
    val_preds.append(val_pred)
    feature_importance_df["Importance_Fold"+str(fold+1)]=model.feature_importance(importance_type='gain')

    # Compute fold metric
    val_pred = pd.DataFrame(data={'prediction': val_pred})
    y_val = pd.DataFrame(data={'target': y_val.reset_index(drop=True)})
    auc_score = roc_auc_score(y_val, val_pred)

    print(f'Fold {fold+1} CV result')
    print(f' ROC metric : {auc_score}')

    del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
    _ = gc.collect()

# Compute out of folds metric
oof_predictions = pd.DataFrame(data={'prediction': oof_predictions})
y_true = pd.DataFrame(data={Config.target: df_train[Config.target]})

print(' ')
print('-'*50)
print(f'TOTAL AUC socre : {roc_auc_score(df_train[Config.target], oof_predictions["prediction"])}')
print('-'*50)

# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame({Config.row_id: df_train[Config.row_id], Config.target: df_train[Config.target], 'prediction': oof_predictions['prediction']})

# Create a dataframe to store test prediction
test_df = pd.DataFrame({Config.row_id: df_test[Config.row_id], Config.target: test_predictions})

 
--------------------------------------------------
Training fold 1 with 32 features...
[LightGBM] [Info] Number of positive: 4429, number of negative: 16376
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002554 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4179
[LightGBM] [Info] Number of data points in the train set: 20805, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.212882 -> initscore=-1.307643
[LightGBM] [Info] Start training from score -1.307643
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[159]	train's binary_logloss: 0.490114	valid's binary_logloss: 0.511104
================================== training 1 fin. ==================================
================================== validation-data predicting ... ===============================

In [ ]:
# Save results
# oof_df.to_csv(f'/content/drive/MyDrive/Amex/OOF/oof_lgbm_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
# test_df.to_csv(f'/content/drive/MyDrive/Amex/Predictions/test_lgbm_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)

In [12]:
oof_df.head()

,id,failure,prediction
0,0,0.0,0.150013
1,1,0.0,0.157230
2,2,0.0,0.152383
3,3,0.0,0.207316
4,4,0.0,0.208557


In [13]:
def plot_roc(y_val, y_prob):
    #colors=px.colors.qualitative.Prism
    fig = go.Figure(layout=plotly_template['layout'])
    fig.add_trace(go.Scatter(x=np.linspace(0,1,11), y=np.linspace(0,1,11), name='Random Chance', mode='lines', showlegend=False, line=dict(color="Black", width=1, dash="dot")))

    for i in range(len(y_val)):
        y=y_val[i]
        prob=y_prob[i]
        fpr, tpr, _ = roc_curve(y, prob)
        roc_auc = auc(fpr,tpr)
        fig.add_trace(go.Scatter(x=fpr, y=tpr, line=dict(color=color_palette['Cat5'][i], width=3),
                                 hovertemplate = 'True positive rate = %{y:.3f}<br>False positive rate = %{x:.3f}',
                                 name='Fold {}: AUC = {:.3f}'.format(i+1, roc_auc)))

    fig.update_layout(template=plotly_template, title="Cross-Validation ROC Curves",
                      hovermode="x unified", width=700, height=600,
                      xaxis_title='False Positive Rate (1 - Specificity)',
                      yaxis_title='True Positive Rate (Sensitivity)',
                      legend=dict(orientation='v', y=.07, x=1, xanchor="right",
                                  bordercolor="black", borderwidth=.5))
    fig.show()

plot_roc(y_valids, val_preds)

In [14]:
top = 50

feature_importance_df['avg'] = feature_importance_df.mean(axis=1)
feature_importance_top = feature_importance_df.avg.nlargest(top).sort_values(ascending=True)

pal=sns.color_palette("YlGnBu", 65).as_hex()
fig=go.Figure()
for i in range(len(feature_importance_top.index)):
    fig.add_shape(dict(type="line", y0=i, y1=i, x0=0, x1=feature_importance_top[i],
                       line_color=pal[::-1][i],opacity=0.8,line_width=4))

fig.add_trace(go.Scatter(x=feature_importance_top, y=feature_importance_top.index, mode='markers',
                         marker_color=pal[::-1], marker_size=8,
                         hovertemplate='%{y} Importance = %{x:.0f}<extra></extra>'))

fig.update_layout(template=plotly_template,title=f'LGBM Feature Importance<br>Top {top}',
                  margin=dict(l=150,t=80),
                  xaxis=dict(title='Importance', zeroline=False),
                  yaxis_showgrid=False, height=1000, width=800)
fig.show()

In [15]:
# test_df = pd.DataFrame({Config.row_id: test[Config.row_id], 'prediction': test_predictions})

df = pd.DataFrame(data={'Target':test_df[Config.target].apply(lambda x: 1 if x>0.25 else 0)})
df = df.Target.value_counts(normalize=True)
df.rename(index={1:'Positive', 0:'Negative'}, inplace=True)

#pal, color=['#016CC9','#DEB078'], ['#8DBAE2','#EDD3B3']
fig=go.Figure()

fig.add_trace(go.Pie(labels=df.index, values=df*100, hole=.45,
                     showlegend=True,sort=False,
                     marker=dict(colors=color_palette['Bin'],line=dict(color=pal,width=2.5)),
                     hovertemplate = "%{label}: %{value:.2f}%<extra></extra>"))

fig.update_layout(template=plotly_template, title='Predicted Target Distribution',
                  legend=dict(traceorder='reversed',y=1.05,x=0),
                  uniformtext_minsize=15, uniformtext_mode='hide',width=700)
fig.show()

In [ ]:
test_df

In [ ]:
test_df[Config.target].describe()

In [16]:
test_df.to_csv(Config.submission_dir + f'nb{Config.NB}.csv', index=False)

## 検証メモ